In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

## 1. Binary Crossentropy

In [2]:
loss_object = BinaryCrossentropy()

# 값의 형태가 단일값인 경우
predictions = np.array([0.3]).reshape(-1, 1)  # 예측값 : 0.3
labels = np.array([1])  # 정답 : 1

In [3]:
loss = loss_object(labels, predictions)
loss_manual = -1*(labels*np.log(predictions) + \
                  (1-labels)*np.log(1-predictions))  # 직접 계산
print(loss.numpy())
print(loss_manual)

1.2039724588394165
[[1.2039728]]


In [18]:
# 값의 형태가 one-hot vector인 경우
predictions = np.array([0.3, 0.7]).reshape(-1, 1)
labels = np.array([1, 0]).reshape(-1, 1)

loss = loss_object(labels, predictions)  # BinaryCrossentropy를 이용할 경우 동일하게 적용

loss_manual = -1*(labels*np.log(predictions)) # 직접 계산할 경우
loss_manual = np.sum(loss_manual)   

print(loss.numpy())
print(loss_manual)

1.2039724588394165
1.2039728043259361


In [11]:
# 여러 개의 데이터가 들어있는 경우(batch 형태)
predictions = np.array([[0.3, 0.7], [0.4, 0.6], [0.1, 0.9]])
labels = np.array([[0, 1], [1, 0], [1, 0]])

loss = loss_object(labels, predictions)
print(loss)

tf.Tensor(1.1918498277664185, shape=(), dtype=float64)


In [14]:
loss_manual = -1*labels*np.log(predictions)
print(loss_manual, '\n')

loss_manual = np.sum(loss_manual, axis=1)
print(loss_manual, '\n')

loss_manual = np.mean(loss_manual)
print(loss_manual)

[[-0.          0.35667494]
 [ 0.91629073 -0.        ]
 [ 2.30258509 -0.        ]] 

[0.35667494 0.91629073 2.30258509] 

1.1918502562689777


## 2. Categorical Crossentropy

In [19]:
loss_object = CategoricalCrossentropy()

predictions = np.array([[0.2, 0.1, 0.7], [0.4, 0.3, 0.3], [0.1, 0.8, 0.1]])
labels = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])

print(predictions)
print(labels)

[[0.2 0.1 0.7]
 [0.4 0.3 0.3]
 [0.1 0.8 0.1]]
[[0 0 1]
 [0 1 0]
 [1 0 0]]


In [25]:
loss = loss_object(labels, predictions)
print(loss.numpy())

1.2877442836761475


In [26]:
loss_manual = -1*labels*np.log(predictions)
loss_manual = np.sum(loss_manual, axis=1)
loss_manual = np.mean(loss_manual)

print(loss_manual)

1.2877442804195713


## 3. Sparse Categorical Crossentropy

In [28]:
# Sparse : 정답값으로 one-hot vector가 아닌, 레이블 값으로 주어지는 경우 사용
loss_object = SparseCategoricalCrossentropy()

predictions = np.array([[0.2, 0.1, 0.7], [0.4, 0.3, 0.3], [0.1, 0.8, 0.1]])
labels = np.array([2, 1, 0])

loss = loss_object(labels, predictions)
print(loss.numpy())

1.2877442836761475


In [29]:
ce_loss = 0
for data_idx in range(len(labels)):
    prediction = predictions[data_idx]
    label = labels[data_idx]

    t_prediction = prediction[label]
    ce_loss += -1*np.log(t_prediction)
ce_loss = ce_loss / len(labels)

print(ce_loss)

1.2877442804195713


## 4. tfds에 적용

In [30]:
import tensorflow_datasets as tfds

train_ds = tfds.load(name='mnist',
                     shuffle_files=True,
                     as_supervised=True,
                     split='train')

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [32]:
train_ds = train_ds.batch(8)
train_ds_iter = iter(train_ds)
images, labels = next(train_ds_iter)

labels

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([4, 1, 0, 7, 8, 1, 2, 7])>

정답값이 레이블 값으로 주어져 있음, Sparse를 사용해야 함!

## 5. Categorical Accuracy

In [35]:
metric = CategoricalAccuracy()

predictions = np.array([[0.2, 0.1, 0.7], [0.4, 0.3, 0.3], [0.1, 0.8, 0.1]])
labels = np.array([[0, 0, 1], [1, 0, 0], [1, 0, 0]])

In [36]:
acc = metric(labels, predictions)
print(acc)

tf.Tensor(0.6666667, shape=(), dtype=float32)


## 6. Sparse Categorical Accuracy

In [37]:
metric = SparseCategoricalAccuracy()

predictions = np.array([[0.2, 0.1, 0.7], [0.4, 0.3, 0.3], [0.1, 0.8, 0.1]])
labels = np.array([2, 0, 0])

In [39]:
acc = metric(labels, predictions)
print(acc*100)

tf.Tensor(66.66667, shape=(), dtype=float32)
